### Import Libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import IsolationForest

### Load Data

In [4]:
df = pd.read_excel(
    'C://Users//MQ745ED//Downloads//Sample Sales Data v1.xlsx',
    skiprows=0,
    sheet_name='Data'
)

### Making a key

In [5]:
df['Key'] = (
    df['Market'].astype(str) + "-" +
    df['Product Category'].astype(str)
)


### Dataframe Division

In [6]:


# Group by 'Key' and count non-zero 'AC' values per group
grouped = df.groupby('Key')['Sales Quantity'].apply(lambda x: (x != 0).sum())

# Get keys where non-zero count < 6
keys_less_than_6 = grouped[grouped < 6].index.tolist()
keys_6_or_more = grouped[grouped >= 6].index.tolist()

# Split into two DataFrames
df_less_than_6 = df[df['Key'].isin(keys_less_than_6)]
df_6_or_more = df[df['Key'].isin(keys_6_or_more)]

# Display counts (optional)
print(f"Groups with <6 non-zero 'Sales Quantity' values: {len(keys_less_than_6)}")
print(f"Groups with >=6 non-zero 'Sales Quantity' values: {len(keys_6_or_more)}")

Groups with <6 non-zero 'AC' values: 7
Groups with >=6 non-zero 'AC' values: 49


In [10]:
df = df_6_or_more.copy()

In [7]:
df_less_than_6.to_excel('C://Users//MQ745ED//OneDrive - EY//SampleCaseStudy//Output//Filtered Dataset Less then 6.xlsx',index = False)

In [8]:
df_6_or_more.to_excel('C://Users//MQ745ED//OneDrive - EY//SampleCaseStudy//Output//Filtered Dataset More then 6 .xlsx',index = False)

### Exploratory Analysis

In [11]:
df['Market'].nunique()

8

In [13]:
df['Product Category'].nunique()

7

In [16]:
df['Key'].nunique()

49

### Check if data is normally distributed

In [17]:
normality_check = df.copy()

In [18]:
import pandas as pd
from scipy.stats import shapiro, skew, kurtosis

# Ensure AC column is numeric
normality_check['Sales Quantity'] = pd.to_numeric(normality_check['Sales Quantity'], errors='coerce')

# Function to check normality and assign the same results to each row in the group
def check_normality(group):
    ac_vals = group['Sales Quantity'].dropna()
    
    if len(ac_vals) >= 3:  # Shapiro needs at least 3 points
        stat, p_val = shapiro(ac_vals)
        is_norm = p_val > 0.05
        skew_val = skew(ac_vals)
        kurt_val = kurtosis(ac_vals)
    else:
        is_norm = None
        p_val = None
        skew_val = None
        kurt_val = None

    group['is_normal'] = is_norm
    group['p_value'] = p_val
    group['skewness'] = skew_val
    group['kurtosis'] = kurt_val
    return group

# Apply to each KPI Code and Plant Name combination
normality_check = normality_check.groupby(['Market', 'Product Category'], group_keys=False).apply(check_normality)


C:\Users\MQ745ED\AppData\Local\Temp\ipykernel_7752\652310773.py:29: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  normality_check = normality_check.groupby(['Market', 'Product Category'], group_keys=False).apply(check_normality)


### Getting outliers using IQR

In [19]:
def detect_outliers(group):
    if len(group) < 3:
        group['Q1'] = group['Sales Quantity'].quantile(0.25)
        group['Q3'] = group['Sales Quantity'].quantile(0.75)
        group['IQR'] = group['Q3'] - group['Q1']
        group['is_outlier'] = False
    else:
        Q1 = group['Sales Quantity'].quantile(0.25)
        Q3 = group['Sales Quantity'].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        group['Q1'] = Q1
        group['Q3'] = Q3
        group['IQR'] = IQR
        group['is_outlier'] = (group['Sales Quantity'] < lower_bound) | (group['Sales Quantity'] > upper_bound)

    return group


In [20]:
outliers_std_df = normality_check.groupby(['Market', 'Product Category'], group_keys=False).apply(detect_outliers)


C:\Users\MQ745ED\AppData\Local\Temp\ipykernel_7752\1000332750.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  outliers_std_df = normality_check.groupby(['Market', 'Product Category'], group_keys=False).apply(detect_outliers)


### Calculate Impact on Mean for each line item

In [23]:
# Calculate mean of 'Sales Quantity' only where 'is_outlier' is False, per group
outliers_std_df['Sum without outliers'] = (
    outliers_std_df.groupby('Key')['Sales Quantity']
    .transform(lambda x: x[~outliers_std_df.loc[x.index, 'is_outlier']].sum())
)

In [24]:
# Calculate mean of 'Sales Quantity' only where 'is_outlier' is False, per group
outliers_std_df['Count without outliers'] = (
    outliers_std_df.groupby('Key')['Sales Quantity']
    .transform(lambda x: x[~outliers_std_df.loc[x.index, 'is_outlier']].count())
)

In [26]:
# Calculate mean of 'Sales Quantity' only where 'is_outlier' is False, per group
outliers_std_df['Mean without outliers'] = (
    outliers_std_df.groupby('Key')['Sales Quantity']
    .transform(lambda x: x[~outliers_std_df.loc[x.index, 'is_outlier']].mean())
)

In [27]:
outliers_std_df['Mean without outliers'] = outliers_std_df['Mean without outliers'].astype(float)
# Calculate for rows where is_outlier is False
outliers_std_df['New Mean outliers'] = np.where(
    outliers_std_df['is_outlier'],
    (outliers_std_df['Sum without outliers'] + outliers_std_df['Sales Quantity']) / 
    (outliers_std_df['Count without outliers'] + 1),
    0.0  # Value when is_outlier is True
)

outliers_std_df['New Mean outliers'] = outliers_std_df['New Mean outliers'].astype(float)
outliers_std_df['Impact on Mean'] = np.where(
    outliers_std_df['is_outlier'],
    ((outliers_std_df['New Mean outliers'] - outliers_std_df['Mean without outliers']) / 
     outliers_std_df['Mean without outliers']) * 100,
    0.0
)

### Save File

In [28]:
outliers_std_df.to_excel('C://Users//MQ745ED//OneDrive - EY//SampleCaseStudy//Output////Outlier Detected.xlsx',index = False)

In [29]:
import pandas as pd
from prophet import Prophet


C:\Users\MQ745ED\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [31]:
# 1. Load data
file_path = "C://Users//MQ745ED//OneDrive - EY//SampleCaseStudy//Output////Outlier Detected.xlsx"
df = pd.read_excel(file_path)

In [32]:
df['Date'] = pd.to_datetime(df['Year'])
df = df.sort_values(by=['Market', 'Product Category', 'Date'])


In [33]:
df['Date']

258   1970-01-01 00:00:00.000002019
259   1970-01-01 00:00:00.000002020
260   1970-01-01 00:00:00.000002021
261   1970-01-01 00:00:00.000002022
262   1970-01-01 00:00:00.000002023
                   ...             
67    1970-01-01 00:00:00.000002020
68    1970-01-01 00:00:00.000002021
69    1970-01-01 00:00:00.000002022
70    1970-01-01 00:00:00.000002023
71    1970-01-01 00:00:00.000002024
Name: Date, Length: 294, dtype: datetime64[ns]